In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor, StackingRegressor

## file processing and feature engineering

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#load dataset
flights = pd.read_csv('flights_raw.txt')

#feature engineering
flights['fl_date'] = pd.to_datetime(flights['fl_date'], format='%Y-%m-%d')
flights['year'] = flights['fl_date'].dt.year
flights['month'] = flights['fl_date'].dt.month
flights['day'] = flights['fl_date'].dt.day
flights['arr_delay'] = flights['arr_delay'].fillna(0)
flights = flights.dropna()

#Dropping the outlier rows with standard deviation
factor = 3
upper_lim = flights['arr_delay'].mean() + flights['arr_delay'].std() * factor
lower_lim = flights['arr_delay'].mean() - flights['arr_delay'].std() * factor

flights = flights[(flights['arr_delay'] < upper_lim) & (flights['arr_delay'] > lower_lim)]

#binning distance
flights['bin_dist'] = pd.cut(flights['distance'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=[1,2,3,4,5,6,7,8,9,10])

#binning time
flights['bin_time'] = pd.cut(flights['crs_elapsed_time'], bins=[0,120,240,360,480,600,2000], labels=[1,2,3,4,5,6])

#log of distance
flights['log_dist'] = (flights['distance']+1).transform(np.log)

#log of elapsed time
flights['log_time'] = (flights['crs_elapsed_time']+1).transform(np.log)

#one-hot encoding for op_unique_carrier
encoded_columns = pd.get_dummies(flights['op_unique_carrier'])
flights = flights.join(encoded_columns).drop('op_unique_carrier', axis=1)

flights.head()

,row_num,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,year,month,day,bin_dist,bin_time,log_dist,log_time,9E,9K,AA,AS,AX,B6,C5,CP,DL,EM,EV,F9,G4,G7,HA,KS,MQ,NK,OH,OO,PT,QX,UA,VX,WN,YV,YX,ZW
0,300,2018-01-01,B6,B6,B6,523,N984JB,523,12478,JFK,"New York, NY",12892,LAX,"Los Angeles, CA",1235,1550,N,375,1,2475,-6.0,2018,1,1,5,4,7.814400,5.929589,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,600,2018-01-01,B6,B6,B6,1224,N969JB,1224,12892,LAX,"Los Angeles, CA",12478,JFK,"New York, NY",1700,110,N,310,1,2475,17.0,2018,1,1,5,3,7.814400,5.739793,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,900,2018-01-01,HA,HA,HA,16,N373HA,16,12173,HNL,"Honolulu, HI",14679,SAN,"San Diego, CA",1505,2220,N,315,1,2614,44.0,2018,1,1,6,3,7.869019,5.755742,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1200,2018-01-01,UA,UA_CODESHARE,UA,3443,N979RP,3443,11292,DEN,"Denver, CO",14457,RAP,"Rapid City, SD",1130,1249,N,79,1,300,-7.0,2018,1,1,1,1,5.707110,4.382027,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1500,2018-01-01,AA,AA,AA,608,N742PS,608,11298,DFW,"Dallas/Fort Worth, TX",10397,ATL,"Atlanta, GA",1645,1948,N,123,1,731,-7.0,2018,1,1,2,2,6.595781,4.820282,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
# add test file
test_df = pd.read_csv('test-jan-1_7.csv')
test_df.head(3)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,day
0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,1
1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363,1
2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333,1


## processing the test file

In [36]:
# feature engineer test file

#feature engineering
test_df['fl_date'] = pd.to_datetime(test_df['fl_date'], format='%Y-%m-%d')
test_df['year'] = test_df['fl_date'].dt.year
test_df['month'] = test_df['fl_date'].dt.month
test_df['day'] = test_df['fl_date'].dt.day

#binning distance
test_df['bin_dist'] = pd.cut(test_df['distance'], bins=[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000], labels=[1,2,3,4,5,6,7,8,9,10])

#binning time
test_df['bin_time'] = pd.cut(test_df['crs_elapsed_time'], bins=[0,120,240,360,480,600,2000], labels=[1,2,3,4,5,6])

#log of distance
test_df['log_dist'] = (test_df['distance']+1).transform(np.log)

#log of elapsed time
test_df['log_time'] = (test_df['crs_elapsed_time']+1).transform(np.log)

#one-hot encoding for op_unique_carrier
encoded_columns = pd.get_dummies(test_df['op_unique_carrier'])
test_df = test_df.join(encoded_columns).drop('op_unique_carrier', axis=1)

test_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,day,day_of_week,year,month,bin_dist,bin_time,log_dist,log_time,9E,AA,AS,AX,B6,C5,CP,DL,EM,EV,F9,G4,G7,HA,MQ,NK,OH,OO,PT,QX,UA,WN,YV,YX,ZW
0,2020-01-01,WN,WN,WN,5888,N951WN,5888,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363,1,2,2020,1,1,1,5.897154,4.564348,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2020-01-01,WN,WN,WN,6276,N467WN,6276,13891,ONT,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363,1,2,2020,1,1,1,5.897154,4.510860,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2020-01-01,WN,WN,WN,4598,N7885A,4598,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333,1,2,2020,1,1,1,5.811141,4.262680,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,2020-01-01,WN,WN,WN,4761,N551WN,4761,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333,1,2,2020,1,1,1,5.811141,4.330733,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,2020-01-01,WN,WN,WN,5162,N968WN,5162,13891,ONT,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333,1,2,2020,1,1,1,5.811141,4.394449,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [34]:
# strip relevant columns from test file
feat = ['op_unique_carrier', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'day_of_week']
test = test_df[feat]
test.shape

(150623, 6)

## validation

In [42]:
# Split-out validation dataset
features = ['origin_airport_id','dest_airport_id','crs_dep_time','log_time','month','log_dist','9E','AA','AS','AX','B6','C5','CP','DL','EM','EV','F9','G4','G7','HA','MQ','NK','OH','OO','PT','QX','UA','WN','YV','YX','ZW']
X = flights[features]
y = flights['arr_delay']
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [43]:
test = test_df[features]

In [10]:
flights[features].to_csv('preprocessed_data.csv', index=False)

In [64]:
Y_train = Y_train.values.reshape(-1, 1)
Y_test = Y_test.values.reshape(-1, 1)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((41592, 31), (10398, 31), (41592, 1), (10398, 1))

## final model training and prediction

In [46]:
# add models with determined parameters to list for meta-estimator
estimators = [
    ('trees', ExtraTreesRegressor()),
    ('boost', HistGradientBoostingRegressor(
        warm_start = True
    ))
]

In [47]:
# initialize meta estimator
reg = StackingRegressor(
    estimators = estimators,
    passthrough = True)

In [48]:
# train model on full sampled data
reg.fit(X_train, Y_train)

StackingRegressor(estimators=[('trees', ExtraTreesRegressor()),
                              ('boost',
                               HistGradientBoostingRegressor(warm_start=True))],
                  passthrough=True)

In [49]:
# training score
reg.score(X_train, Y_train)

0.14909606818448629

In [51]:
# predict on submission test data
y_pred = reg.predict(test)
y_pred

array([ 6.01171678,  0.12419833,  8.11235444, ...,  0.56778289,
       -0.82997991, -8.8051679 ])

## create final submission file

In [52]:
# add sample submission file to copy structure
sample = pd.read_csv('mid-term-project-I/sample_submission.csv')
sample.head(3)

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5888,ONT,SFO,6.442385
1,2020-01-01,WN,6276,ONT,SFO,6.174046
2,2020-01-01,WN,4598,ONT,SJC,3.233856


In [53]:
# get column names from sample file
cols = sample.columns.tolist()
cols

['fl_date',
 'mkt_carrier',
 'mkt_carrier_fl_num',
 'origin',
 'dest',
 'predicted_delay']

In [54]:
# remove prediction column name
cols.remove('predicted_delay')

In [56]:
# create submission df from features on submission test file using column names from sample
submission = test_df[cols]
submission.head()

,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest
0,2020-01-01,WN,5888,ONT,SFO
1,2020-01-01,WN,6276,ONT,SFO
2,2020-01-01,WN,4598,ONT,SJC
3,2020-01-01,WN,4761,ONT,SJC
4,2020-01-01,WN,5162,ONT,SJC


In [57]:
# add predictions to submission file
submission['predicted_delay'] = y_pred
submission.head()

C:\Users\jayta\AppData\Local\Temp/ipykernel_1216/1331249918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['predicted_delay'] = y_pred


,fl_date,mkt_carrier,mkt_carrier_fl_num,origin,dest,predicted_delay
0,2020-01-01,WN,5888,ONT,SFO,6.011717
1,2020-01-01,WN,6276,ONT,SFO,0.124198
2,2020-01-01,WN,4598,ONT,SJC,8.112354
3,2020-01-01,WN,4761,ONT,SJC,2.966378
4,2020-01-01,WN,5162,ONT,SJC,-1.074710


In [58]:
# export final submission file
submission.to_csv('submission.csv', index=False)